In [1]:
using DiffEqBase, DiffEqOperators, OrdinaryDiffEq, LinearAlgebra

# TransitionModel defn
struct TransitionModel
    A # transition matrix
    args
    t # current time
end

function (M::TransitionModel)(du, u, p, t)
    LinearAlgebra.mul!(du, M, u)
end

# overload methods for matrix analogy
LinearAlgebra.size(m::TransitionModel, i::Int64) = size(m.A, i)
LinearAlgebra.ishermitian(m::TransitionModel) = ishermitian(m.A)
transition_dynamics!(y, m::TransitionModel, b) = mul!(y, m.A, b)
function transition_dynamics!(Y, m::TransitionModel, B::AbstractMatrix{T}) where {T} 
    N = size(A,1)
    for j in 1:(N+1)
        Y[:,j] = mul!(similar(B[:,j]), m, B[:,j])
    end    
end
LinearAlgebra.opnorm(m::TransitionModel, p) = opnorm(m.A, p)

# solution setup
T = 1
ts = 0:0.1:T
f_0 = [1.0; 0.0; 0.0]

3-element Array{Float64,1}:
 1.0
 0.0
 0.0

### Models with time-independent operators

In [2]:
# model specs
A = [[-0.5 0.25 0.25]; [-0.5 0.0 0.5]; [0.0 -0.5 0.5]]
model = TransitionModel(A, nothing, 0.0)

# solve
prob = ODEProblem(MatrixFreeOperator((du,u)->transition_dynamics!(du, model, u)),f_0,(0.0,T))
solve(prob, LawsonEuler(krylov=true, m=5), dt=0.1)

retcode: Success
Interpolation: 3rd order Hermite
t: 11-element Array{Float64,1}:
 0.0                
 0.1                
 0.2                
 0.30000000000000004
 0.4                
 0.5                
 0.6                
 0.7                
 0.7999999999999999 
 0.8999999999999999 
 1.0                
u: 11-element Array{Array{Float64,1},1}:
 [1.0, 0.0, 0.0]                   
 [0.950635, -0.0487397, 0.00124987]
 [0.902582, -0.0949188, 0.00499792]
 [0.855901, -0.138479, 0.0112395]  
 [0.810649, -0.179367, 0.0199667]  
 [0.766884, -0.217531, 0.0311687]  
 [0.724661, -0.252924, 0.0448315]  
 [0.684031, -0.2855, 0.060938]     
 [0.645046, -0.31522, 0.0794681]   
 [0.607755, -0.342046, 0.100399]   
 [0.572203, -0.365945, 0.123703]   

### Models with time-dependent operators 
(TODO: need more methods to be overridden. See https://github.com/JuliaDiffEq/DiffEqOperators.jl/pull/81)

In [3]:
# # model specs
# A = [[-0.5 0.25 0.25]; [-0.5 0.0 0.5]; [0.0 -0.5 0.5]]
# model = TransitionModel(A, nothing, 0.0)
# 
# # solve
# function get_operator(m::TransitionModel) 
#     function awareness_operator_basis(du,u,p,t)
#         m.t = t
#         transition_dynamics!(du, m, u)
#     end
#     return MatrixFreeOperator(awareness_operator_basis)
# end
# prob = ODEProblem(MatrixFreeOperator((du,u,p,t)->transition_dynamics!(du, model, u), nothing),f_0,(0.0,T) )

# # prob = ODEProblem(get_operator(model),f_0,(0.0,T))
# solve(prob, LawsonEuler(krylov=true, m=5), dt=0.1)